# Final Project: Vehicle Rounting Problem

In [33]:
import numpy as np
#np.set_printoptions(threshold=np.nan)
import seaborn as sns
from matplotlib import pyplot as plt
import math

## 1. Import Data

In [23]:
# VRP1
# import distance matrix
filepath = 'VRP1/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix.append(y)
        
    t.close()
    distance_matrix = np.array(distance_matrix)

# import demand matrix
filepath = 'VRP1/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix.append(y)
        
    t.close()
    demand_matrix = np.array(demand_matrix)

# import capacity matrix
filepath = 'VRP1/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix.append(y)
        
    t.close()
    capacity_matrix = np.array(capacity_matrix)

# VRP1
# import distance matrix
filepath = 'VRP2/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix_2.append(y)
        
    t.close()
    distance_matrix_2 = np.array(distance_matrix_2)

# import demand matrix
filepath = 'VRP2/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix_2.append(y)
        
    t.close()
    demand_matrix_2 = np.array(demand_matrix)

# import capacity matrix
filepath = 'VRP2/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix_2.append(y)
        
    t.close()
    capacity_matrix_2 = np.array(capacity_matrix)

In [24]:
# information about the distance matrix for VRP1
print("Information about distance matric for VRP1 \n# columns: ", len(distance_matrix[0]))
print("# lines: ", len(distance_matrix))
asymmetric = (distance_matrix != distance_matrix.T)
print(np.unique(asymmetric))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric. \n")

# information about the distance matrix for VRP2
print("Information about distance matric for VRP2 \n# columns: ", len(distance_matrix_2[0]))
print("# lines: ", len(distance_matrix_2))
asymmetric_2 = (distance_matrix_2 != distance_matrix_2.T)
print(np.unique(asymmetric_2))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric.")

Information about distance matric for VRP1 
# columns:  101
# lines:  101
[False  True]
Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric. 

Information about distance matric for VRP2 
# columns:  101
# lines:  101
[False  True]
Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric.


In [25]:
#heatmap
ax = sns.heatmap(distance_matrix, center=0)

The heatmap again illustrates the fact that we have to deal with an asymmetric distance matrix. In order to apply the k-means algorithm, we need a symmetric matrix. Therefore we need to make our distances symmetric be taking the mean between the distances (if they are different). 

In [26]:
# CASE 1: distance_matrix 
cleaned_distance_matrix = np.copy(distance_matrix)
for i in range(len(asymmetric)):
    for j in range(i+1, len(asymmetric[0])):
        if asymmetric[i,j]:
            cleaned_distance = (distance_matrix[i,j] + distance_matrix[j,i]) // 2
            cleaned_distance_matrix[i,j] = cleaned_distance
            cleaned_distance_matrix[j,i] = cleaned_distance
            
# CASE 2: distance_matrix_2 
cleaned_distance_matrix_2 = np.copy(distance_matrix_2)
for i in range(len(asymmetric_2)):
    for j in range(i+1, len(asymmetric_2[0])):
        if asymmetric_2[i,j]:
            cleaned_distance = (distance_matrix_2[i,j] + distance_matrix_2[j,i]) // 2
            cleaned_distance_matrix_2[i,j] = cleaned_distance
            cleaned_distance_matrix_2[j,i] = cleaned_distance

In [27]:
# Demand matrix has 100 entries
demand_matrix.shape

# Note that the matrix is 101x101, i.e. that the depot is set at (0)

(1, 100)

## Algorithm

1. Clean matrix, i.e. make the matrix symmetric. Take mean of two different values.
2. K-means clustering to 100-demand-clusters. See Vanessa's paper how to do it. 
3. Look at one 100 cluster and calculate the demand distance ratio: $:= \dfrac{demand}{distance \cdot cost}$ Merge the 100 cluster with the nearest one, calculate new ratio and save it. Calculate new center point. Merge this cluster with the next nearest and calculate ratio, save it, etc. Do this until you reached a demand of 1000. Then Merge the first cluster with the ones that give the best ratio. Go to the next cluster, that hasn't been merged yet and do the same until no more clusters have not been merged or checked. 
4. Apply nature inspired algorithm.


Last idea: 
1. Clean matrix, i.e. make the matrix symmetric. Take mean of two different values.
2. K-means clustering to 100-demand-clusters. See Vanessa's paper how to do it. 
3. Merging: first calculate the demand-distance-ratio for the existing 100-demand-clusters. Then merge 3 100-demand-clusters with K-means clustering into one (do that until all are merged). Calculate demand-distance-ratio of the 300-clusters as well. Note that **demand-distance-ratio** $:= \dfrac{demand}{distance \cdot cost}$
4. Merge all 100-demand-clusters into 500-demand-clusters. Calculate demand-distance-ratio. 
5. Merge all 100-demand-clusters into 1000-demand-clusters. Calculate demand-distance-ratio.
6. Choose cluster with best ratio (smallest value) and mark the customers as final. Check if all customers are marked final. 
7. If not check if we still have all types of cars available. Recalculate the clusters for all non-final clusters for all available car types (repeat step 3-5).

**REMEMBER:** do not hard code parameters, name them 

## 2. Creating coordinate matrix from distance matrix

In [28]:
# input: 101 x 101 matrix (use cleaned_distance_matrix and cleaned_distance_matrix_2)
# depot is node 0 - set it in (0,0)
# output: np.array coordinate_matrix with 2 columns displaying (x,y)-coordinates

## 3. Clustering

In [29]:
# input: distance matrix (use cleaned_distance_matrix and cleaned_distance_matrix_2) and geometrical representation
# of the customers (from point 2), demand_matrix, capacity_matrix

# output: (cluster x customer)-matrix that consists only of 0 and 1 (1 means that the customer is part of the cluster)
# The size of the (cluster x customer)-matrix is approx. 19 x 100
# output: a demand_per_cluster_matrix that consists of the demand per cluster [demand_of_cluster_1, demand_of_cluster_2, ...]

## 4. Merging

In [32]:
# input: (cluster x customer)-matrix: ccmatrix, demand_per_cluster_matrix: dpcmatrix, demand_matrix, cleaned_distance_matrix, 
# capacity_matrix, coordinate_matrix
# 100cluster-centers array:originalclustercenters, 100cluster-intradistances - array:intradistances
# array with car types, number of vehicles per type, cost per type, (one column for type, one for number, one for cost): vehiclematrix

def ddr(demand, distance, cost):
    return demand/(distance*cost)

def distance(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

# prepating mergedclusters    
demand_matrix = np.array([[1,2,3],[3,4,5],[4,5,6]]) #just to test
mergedclusters = np.zeros(len(demand_matrix), dtype=bool)
print(mergedclusters) # to check

# calculate center of 100-clusters - already get it from last method

# get cluster that's farest away from Depot

currentcluster = 0
currentclusterdistance = 0
for(i in range np.size(originalclustercenters,0):
    if((dist(originalclustercenter(i),originalclustercenter(i),coordinate_matrix(0),coordinate_matrix(0)) > currentclusterdistance)&& mergedclusters(i)==False):  #need to access x and y values somehow
        currentcluster = i
        currentclusterdistance = dist((originalclustercenter(i),originalclustercenter(i),coordinate_matrix(0),coordinate_matrix(0))
    

# calculate ddr of cluster, save it in new array: Ratio-array. This is only the code for calculating it for a single 100 cluster, nees to be adapted later. 
                                      
ratioarray = np.array([0])
mergedarraynumber = 0
neighbourarray = np.array([currentcluster])
                                      
                                      
current_cost = vehiclematrix[0,2]
current_distance = intradistances(currentcluster) + 2* dist(originalclustercenter(currentcluster),originalclustercenter(currentcluster),coordinate_matrix(0),coordinate_matrix(0)))
current_ddr = ddr(demand_matrix(currentcluster),current_distance,current_cost)
np.append(ratioarray, current_ddr)
                                          
# find nearest cluster, calculate ddr with fitting car, if they would be merged, save it in Ratio-array
    

    
# calculate new center
# repeat last two steps and stop when demand will be over thousand
# find best merging partners by looking at highest value in Ratio array.
# Merge according clusters.
# set merged clusters to value TRUE in mergedclusters-matrix.

#repeat with next cluster that's farest away, that has the value FALSE in mergedclusters-matrix

# output: final_cluster_customer_matrix

[False False False]


## 5. Differential Evolution/Genetic Algorithm/Ant Algorithm

In [10]:
# input: final_cluster_customer_matrix

## 6. Run

In [38]:
demand_matrix = np.array([[1,2,3],[9,4,5],[7,8,6]]) #just to test
print(demand_matrix[1,2])
print(demand_matrix)

5
[[1 2 3]
 [9 4 5]
 [7 8 6]]
